---
<center>

#  
# **Deep Learning - IBDA3311**
# Proyek UAS - Text Summarizer using Distilled Bart

Arion Ulibasa - **202001287**<br>
Victor Chendra - **202000338**<br>
Victor Chendra - **202000338**<br>
Jason Caleb - **202001240**<br>
Jastin Suriono - **202000162**

</center>

---


In [1]:
import torch
import torch.utils.tensorboard

# tensorboard --logdir=runs

ModuleNotFoundError: No module named 'torch'

# Dataset


In [2]:
!pip install datasets

In [7]:
from datasets import load_dataset

# dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0", cache_dir="./cachedir")
ldtrain = load_dataset(
    "ccdv/cnn_dailymail", "3.0.0", split="train[:2000]", cache_dir="./cachedir"
)
# ldtest = load_dataset("ccdv/cnn_dailymail", "3.0.0", split='test', cache_dir="./cachedir").select(range(500))
# ldvalidation = load_dataset("ccdv/cnn_dailymail", "3.0.0", split='validation', cache_dir="./cachedir").select(range(500))
# dataset

Extracting data files: 100%|██████████| 5/5 [00:00<00:00, 649.25it/s]
Generating train split: 137616 examples [04:59, 53.23 examples/s]  

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 500
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 500
    })
})

# Preprocessing


In [ ]:
import re  # library for regular expression operations
import string  # for string operations

import nltk
from nltk.corpus import stopwords  # module for stop words that come with NLTK
from nltk.stem import PorterStemmer  # module for stemming
from nltk.tokenize import TweetTokenizer  # module for tokenizing strings

from transformers import AutoTokenizer

nltk.download("punkt")
nltk.download("stopwords")
checkpoint = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def preprocess(data, isTrain=True):
    if isTrain:
        # Remove stock market tickers
        data["article"] = [re.sub(r"\$\w*", "", article) for article in data["article"]]
        # Remove hyperlinks
        data["article"] = [
            re.sub(r"https?:\/\/.*[\r\n]*", "", article) for article in data["article"]
        ]
        # Remove hashtags
        data["article"] = [re.sub(r"#", "", article) for article in data["article"]]

    # Tokenize the article
    article_tokens = [
        nltk.tokenize.word_tokenize(article) for article in data["article"]
    ]

    # Remove stopwords and punctuation, and apply stemming
    stemmer = PorterStemmer()
    stopwordsEng = set(stopwords.words("english"))
    article_clean = [
        [
            stemmer.stem(word)
            for word in tokens
            if word.lower() not in stopwordsEng and word not in string.punctuation
        ]
        for tokens in article_tokens
    ]

    # Tokenize words in an article
    joint_articles = [" ".join(cleaned_tokens) for cleaned_tokens in article_clean]

    inputs = tokenizer(
        joint_articles,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512,
    )
    targets = tokenizer(
        data["highlights"],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    inputs["labels"] = targets["input_ids"]
    return inputs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
import torch
from functools import partial
from datasets import DatasetDict

p1 = partial(preprocess, isTrain=True)
p2 = partial(preprocess, isTrain=False)
p3 = partial(preprocess, isTrain=False)

# Apply the preprocess function to the dataset
dptrain = dataset["train"].map(p1, batched=True)  # not yet implement batched
dptest = dataset["test"].map(p2, batched=True)  # not yet implement batched
dpvalidation = dataset["validation"].map(p3, batched=True)  # not yet implement batched

# merge all Dataset into DatasetDict again
prepDataset = DatasetDict(
    {"train": dptrain, "test": dptest, "validation": dpvalidation}
)

# convert to torch, for the purpose of Trainer
columns = ["input_ids", "attention_mask", "labels"]
prepDataset.set_format(type="torch", columns=columns)
prepDataset

Parameter 'function'=functools.partial(<function preprocess at 0x7cf63f91f0a0>, isTrain=True) of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# Model (Victor, Arion) --> Deadline: Kamis 7


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [ ]:
!pip install accelerate==0.20.3
!pip install transformers[torch]
# !pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip freeze

In [ ]:
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from torch.utils.data import DataLoader

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    evaluation_strategy="epoch",
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    remove_unused_columns=False,
    predict_with_generate=True,
    fp16=True,
)

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    train_dataset=prepDataset["train"],
    eval_dataset=prepDataset["validation"],
)

# Fine-tune the model
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.875427
2,2.021000,0.867954
3,2.021000,0.885350


TrainOutput(global_step=750, training_loss=1.5832434895833334, metrics={'train_runtime': 551.1402, 'train_samples_per_second': 10.887, 'train_steps_per_second': 1.361, 'total_flos': 4643736256512000.0, 'train_loss': 1.5832434895833334, 'epoch': 3.0})

In [ ]:
torch.save(model, "./pretrained/ts.pth")

In [ ]:
loaded = torch.load("/content/pretrained/ts.pth")

In [ ]:
loaded.state_dict()

OrderedDict([('final_logits_bias',
              tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')),
             ('model.shared.weight',
              tensor([[-3.6175e-02,  7.8377e-02,  1.6017e-01,  ...,  1.8334e-01,
                        5.8456e-02, -6.5115e-02],
                      [ 8.2750e-03,  8.5320e-04, -4.7186e-02,  ..., -6.9267e-03,
                        6.6192e-03,  5.3263e-03],
                      [-4.7032e-02,  4.5659e-01, -6.4041e-02,  ...,  1.0700e-01,
                        3.4057e-02,  4.8476e-02],
                      ...,
                      [-2.6318e-02,  1.7940e-02, -4.1703e-02,  ..., -1.0457e-02,
                       -5.6920e-02, -2.8223e-02],
                      [-1.9668e-02,  6.1361e-03, -4.3412e-02,  ...,  2.3931e-02,
                       -1.6640e-02,  4.3547e-03],
                      [-1.3936e-02,  6.0645e-03, -5.1557e-02,  ...,  3.1558e-02,
                        5.7227e-03, -3.0961e-04]], device='cuda:0')),
             ('model.

In [ ]:
model.state_dict()

OrderedDict([('final_logits_bias',
              tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')),
             ('model.shared.weight',
              tensor([[-3.6175e-02,  7.8377e-02,  1.6017e-01,  ...,  1.8334e-01,
                        5.8456e-02, -6.5115e-02],
                      [ 8.2750e-03,  8.5320e-04, -4.7186e-02,  ..., -6.9267e-03,
                        6.6192e-03,  5.3263e-03],
                      [-4.7032e-02,  4.5659e-01, -6.4041e-02,  ...,  1.0700e-01,
                        3.4057e-02,  4.8476e-02],
                      ...,
                      [-2.6318e-02,  1.7940e-02, -4.1703e-02,  ..., -1.0457e-02,
                       -5.6920e-02, -2.8223e-02],
                      [-1.9668e-02,  6.1361e-03, -4.3412e-02,  ...,  2.3931e-02,
                       -1.6640e-02,  4.3547e-03],
                      [-1.3936e-02,  6.0645e-03, -5.1557e-02,  ...,  3.1558e-02,
                        5.7227e-03, -3.0961e-04]], device='cuda:0')),
             ('model.

# Analysis (Jastin) --> Deadline:


1.  heatmap
2.  epoch stat
3.  training accuracy n validation accuracy (


# Evaluation (Bang Halel) --> Deadline:


rouge 2/rouge L


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from rouge_score import rouge_scorer

# Load the validation dataset
validation_dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="validation")

# Define the T5 model and tokenizer
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


# Define the summarizer function
def summarizer(input_text, max_length=100, min_length=30):
    input_ids = tokenizer.encode(
        input_text, return_tensors="pt", max_length=1024, truncation=True
    )

    # Generate summary
    summary_ids = model.generate(
        input_ids,
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        early_stopping=True,
    )

    # Decode the generated summary
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return generated_summary


# Function to calculate ROUGE scores
def calculate_rouge_scores(dataset):
    rouge_scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"], use_stemmer=True
    )
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for example in dataset:
        # Generate summary using the summarizer function
        generated_summary = summarizer(example["article"])

        # Calculate ROUGE scores between generated summary and reference summary
        rouge_scores = rouge_scorer.score(example["highlights"], generated_summary)

        rouge1_scores.append(rouge_scores["rouge1"].fmeasure)
        rouge2_scores.append(rouge_scores["rouge2"].fmeasure)
        rougeL_scores.append(rouge_scores["rougeL"].fmeasure)

    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

    print(f"Average ROUGE-1 Score: {avg_rouge1:.4f}")
    print(f"Average ROUGE-2 Score: {avg_rouge2:.4f}")
    print(f"Average ROUGE-L Score: {avg_rougeL:.4f}")


# Evaluate the model using ROUGE scores on the validation set
calculate_rouge_scores(validation_dataset)

# Question

- Why use padding? Is it necessary for text summarization? <br>

  - Memory state: LSTMs maintain an internal memory state that carries information across time steps. When inputs have different lengths, the memory state becomes <u>inconsistent and the network cannot effectively learn long-term dependencies</u>. This can significantly impact the performance of the LSTM model.
  - Vectorization: This is the process of converting loops into vector operations. LSTMs rely heavily on vectorized matrix operations for efficient training and inference. When inputs have different lengths, it becomes difficult to represent them in a single matrix for these operations. This can lead to inefficient computation and slower training times.

- How to use batch when preprocessing (.map)?
- Is it really unnecessary to modify the pretrained model?
- List item
